# Breakdown per NOUNs

Study single video retrieval results per NOUNs.

- Use cell below to dump CSV and add data to G-sheets. To quickly swift through data and plot results.

In [3]:
import numpy as np
import pandas as pd
import json

yfcc100m_csv = '../data/interim/yfcc100m/001.csv'
nouns_to_video_didemo_json = '../data/interim/didemo/nouns_to_video.json'
exp_csv = '../data/interim/hsmcn_07/2.csv'
OUTPUT_FILE = '../data/interim/hsmcn_07/2_breakdown.csv'
SUBSET = 'val'
METRICS = ['iou', 'r@1', 'r@5']

yfcc100m_df = pd.read_csv(yfcc100m_csv)
with open(nouns_to_video_didemo_json, 'r') as fid:
    nouns_to_instances = json.load(fid)['annotations_per_subset'][SUBSET]
exp_df = pd.read_csv(exp_csv)

nouns_metrics = []
underrep_ids = []
for tag, _ in yfcc100m_df.groupby('topk_tags'):
    annotation_ids = nouns_to_instances[tag]
    underrep_ids.extend(annotation_ids)
    ind = np.in1d(exp_df['annotation_id'], annotation_ids)
    if len(ind) == 0:
        continue
    # groupby if slow as hell
    metrics = [exp_df.loc[ind, i].mean() for i in METRICS]
    nouns_metrics.append([tag] + metrics)

# Add results from under-represented NOUNs
ind = np.in1d(exp_df['annotation_id'], underrep_ids)
metrics = [exp_df.loc[ind, i].mean() for i in METRICS]
nouns_metrics.append(['NOUNs Underrepresented/Unseen'] + metrics)
wellrep_ids = np.setdiff1d(exp_df['annotation_id'], underrep_ids)
assert set(underrep_ids).isdisjoint(wellrep_ids.tolist())
ind = np.in1d(exp_df['annotation_id'], wellrep_ids)
metrics = [exp_df.loc[ind, i].mean() for i in METRICS]
nouns_metrics.append(['NOUNs Overrepresented'] + metrics)

df = pd.DataFrame(nouns_metrics, columns=['attribute'] + METRICS)
df.to_csv(OUTPUT_FILE, index=None)